In [35]:
import os
import sys
sys.path.append('../Module')
folder_path = '../Dataset'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn import set_config

from loader_csv import charger_csv
from data_to_csv import df_to_csv
from utils import generate_description
from currency import currency_cleaning
from naics_engineering import naicsEngineering

# Spécifiez les chemins des quatre parties du CSV découpé
partie1 = "../Dataset/SBAnational_part_part1.csv"
partie2 = "../Dataset/SBAnational_part_part2.csv"
partie3 = "../Dataset/SBAnational_part_part3.csv"
partie4 = "../Dataset/SBAnational_part_part4.csv"

# Utilisez la fonction charger_csv pour obtenir un DataFrame avec toutes les données réassemblées
df = charger_csv(partie1, partie2, partie3, partie4)


/home/utilisateur/projet/Brief_Prêt/Model_pret/Model/../Module/loader_csv.py:9: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_partie4 = pd.read_csv(partie4)


In [36]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,LoanNr_ChkDgt,int64,0,0.00,899164,"[7368464004, 5483314001, 7666083010, 181163500..."
1,Name,object,14,0.00,779583,"[Una Auto Inc., F & F DRUGS, A & N CONSTRUCTIO..."
2,City,object,30,0.00,32581,"[RIGBY, MONROE, Twin Falls, ORANGE GROVE, OSCE..."
3,State,object,14,0.00,51,"[GA, WA, MN, NY, PA]"
4,Zip,int64,0,0.00,33611,"[84029, 77026, 11238, 8742, 32174]"
5,Bank,object,1559,0.17,5802,"[SELF-HELP CU, UNITED CENTRAL BANK, NORTHWEST ..."
6,BankState,object,1566,0.17,56,"[CA, ME, NY, MN, UT]"
7,NAICS,int64,0,0.00,1312,"[722211, 532230, 236118, 0, 488490]"
8,ApprovalDate,object,0,0.00,9859,"[27-Apr-05, 7-Sep-95, 18-Dec-98, 14-Jul-05, 30..."
9,ApprovalFY,object,0,0.00,71,"[1990, 1995, 2001, 2005, 2003]"


In [37]:
df.shape

(899164, 27)

In [38]:
df = df[df.NAICS != 0]
df['NAICS'] = df['NAICS'].apply(naicsEngineering)
df.NAICS.value_counts()

NAICS
44-45    127251
81        72618
54        68170
31-33     68029
72        67600
23        66646
62        55366
42        48743
56        32685
48-49     22531
71        14640
53        13632
51        11379
52         9496
11         9005
61         6425
21         1851
22          663
55          257
92          229
Name: count, dtype: int64

In [39]:
df = df[(df.NewExist == 1.0) | (df.NewExist == 2.0)]
df.NewExist.value_counts()

NewExist
1.0    502830
2.0    193520
Name: count, dtype: int64

In [40]:
df.shape

(696350, 27)

In [41]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

In [42]:
samples_bad_lol = df[(df['ChgOffDate'].notna())]
df.loc[samples_bad_lol.index, 'MIS_Status'] = samples_bad_lol['MIS_Status'].replace(['P I F', np.nan], 'CHGOFF')
samples_bad_lol['MIS_Status'].value_counts()

MIS_Status
CHGOFF    140687
P I F       4406
Name: count, dtype: int64

In [43]:
status_counts = df.loc[samples_bad_lol.index, 'MIS_Status'].value_counts()#.isna().sum() == 0

print(status_counts)

MIS_Status
CHGOFF    145262
Name: count, dtype: int64


In [44]:
samples_bad = df[(df['ChgOffPrinGr'].notna()) & ((df['ChgOffPrinGr'] != 0.0) | (df['ChgOffPrinGr'] > 0.0))]
df.loc[samples_bad.index, 'MIS_Status'] = samples_bad['MIS_Status'].replace(['P I F', np.nan], 'CHGOFF')
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).fillna(0.0).astype(float).astype(int)
samples_bad['MIS_Status'].value_counts()

MIS_Status
CHGOFF    145148
Name: count, dtype: int64

In [45]:
samples_bad['MIS_Status'].isna().sum()

0

In [46]:
status_counts = df.loc[samples_bad.index, 'MIS_Status'].value_counts()#.isna().sum() == 0

status_counts

MIS_Status
CHGOFF    145148
Name: count, dtype: int64

In [47]:
mapping = {'0': 'N', '1': 'Y'}
df['RevLineCr'] = df['RevLineCr'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['RevLineCr'].isin(valid_values)]

df['RevLineCr'].value_counts()

RevLineCr
N    484187
Y    195147
Name: count, dtype: int64

In [48]:
mapping = {'0': 'N', '1': 'Y'}
df['LowDoc'] = df['LowDoc'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['LowDoc'].isin(valid_values)]

df['LowDoc'].value_counts()

LowDoc
N    608823
Y     66203
Name: count, dtype: int64

In [49]:
df.dropna(subset=['BankState', 'Bank', 'State'], inplace=True)

In [50]:
df.drop_duplicates(subset=['LoanNr_ChkDgt'], inplace=True)

In [51]:
df['FranchiseCode'] = df['FranchiseCode'].apply(lambda x: 1 if x >= 1 else 0)

In [52]:
df['FranchiseCode'].value_counts()

FranchiseCode
1    493367
0    181033
Name: count, dtype: int64

In [53]:
MIS_Status = pd.get_dummies(df['MIS_Status'], drop_first=True)
df.drop(['MIS_Status'], axis=1, inplace=True)
df = pd.concat([df, MIS_Status], axis=1)
df = df.rename(columns={'P I F': 'Approve'})

In [54]:
leaking_feature = ['ChgOffPrinGr', 'BalanceGross', 'DisbursementGross', 'DisbursementDate', 'ChgOffDate']
useless_feature = ['Name', 'Zip', 'LoanNr_ChkDgt', 'City', 'State', 'Bank', 'BankState', 'ApprovalDate', 'ApprovalFY']
df = df.drop(leaking_feature, axis=1)
df = df.drop(useless_feature, axis=1)
df.dropna(inplace=True)

In [55]:
df.shape

(674400, 13)

In [56]:
df.head()

,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,GrAppv,SBA_Appv,Approve
0,44-45,84,4,2.0,0,0,1,0,N,Y,60000,48000,True
1,72,60,2,2.0,0,0,1,0,N,Y,40000,32000,True
2,62,180,7,1.0,0,0,1,0,N,N,287000,215250,True
5,31-33,120,19,1.0,0,0,1,0,N,N,517000,387750,True
7,81,84,1,2.0,0,0,1,0,N,Y,45000,36000,True


In [57]:
df_to_csv(df, folder_path, 'SBA_Cleaned_maybe.csv')

Le DataFrame a été converti avec succès en fichier CSV : ../Dataset/SBA_Cleaned_maybe.csv


In [58]:
y = df['Approve']
X = df.drop('Approve', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.05, random_state=42, stratify=y)

In [59]:
numerical_columns_selector = X.select_dtypes(exclude='object').columns
categorical_columns_selector = X.select_dtypes(include='object').columns

numerical_columns = X[numerical_columns_selector]
categorical_columns = X[categorical_columns_selector]

In [ ]:
categorical_preprocessor = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
numerical_preprocessor = StandardScaler()

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_preprocessor, numerical_columns_selector),
        ("cat", categorical_preprocessor, categorical_columns_selector)
    ]
)

In [60]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', RandomForestClassifier(n_jobs=-1, random_state=42))
])

set_config(transform_output="pandas")


Model Accuracy: 0.908


In [ ]:
param_grid = {
    'model__n_estimators': [50, 100, 150],
    'model__max_depth': [None, 10, 20, 30],
    'model__min_samples_split': [2, 5, 10],
    'model__min_samples_leaf': [1, 2, 4],
    'model__max_features': ['auto', 'sqrt', 'log2']
}

In [ ]:
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1_macro')
grid_search.fit(X_train, y_train)

In [ ]:
print("Meilleurs hyperparamètres trouvés :")
print(grid_search.best_params_)

print("Meilleure précision trouvée :", grid_search.best_score_)

In [ ]:
best_model = grid_search.best_estimator_

predictions = best_model.predict(X_test)

In [ ]:
f1_macro = f1_score(y_test, predictions, average='macro')

print("F1-score macro sur l'ensemble de test :", f1_macro)

In [61]:
maybe_leaking_feature = ['RetainedJob', 'CreateJob']
maybe_leaking_fea = ['SBA_Appv', 'GrAppv']
df = df.drop(maybe_leaking_feature, axis=1)

In [62]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,NAICS,object,0,0.0,20,"[23, 44-45, 72, 81]"
1,Term,int64,0,0.0,377,"[63, 60, 64, 108, 84]"
2,NoEmp,int64,0,0.0,499,"[1, 2, 5]"
3,NewExist,float64,0,0.0,2,[1.0]
4,FranchiseCode,int64,0,0.0,2,"[1, 0]"
5,UrbanRural,int64,0,0.0,3,"[1, 0]"
6,RevLineCr,object,0,0.0,2,"[N, Y]"
7,LowDoc,object,0,0.0,2,[N]
8,GrAppv,int64,0,0.0,16868,"[850000, 200000, 20000, 10000]"
9,SBA_Appv,int64,0,0.0,28200,"[3500, 100000, 5000, 12500, 42500]"


In [63]:
y = df['Approve']
X = df.drop('Approve', axis=1)

In [64]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.05, random_state=42, stratify=y)

In [65]:
numerical_columns_selector = X.select_dtypes(exclude='object').columns
categorical_columns_selector = X.select_dtypes(include='object').columns

numerical_columns = X[numerical_columns_selector]
categorical_columns = X[categorical_columns_selector]

In [66]:
categorical_preprocessor = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
numerical_preprocessor = StandardScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ("nums", numerical_preprocessor, numerical_columns_selector),
        ("cat", categorical_preprocessor, categorical_columns_selector)
    ]
)

In [67]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', RandomForestClassifier(max_depth=25, min_samples_leaf=80, min_samples_split=3, n_estimators=70, max_samples=0.7, n_jobs=-1 ,random_state=42))
])

In [68]:
set_config(transform_output="pandas")

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Model Accuracy: {accuracy:.3f}')

f1_macro = f1_score(y_test, y_pred, average='macro')

print("F1-score macro sur l'ensemble de test :", f1_macro)

Model Accuracy: 0.911
